In [ ]:
import geopandas as gpd
import json
import altair as alt
import pandas as pd

In [ ]:
departement_to_region = {'971': '01', '972': '02', '973': '03', '974': '04', '01': '84', '02': '32', '03': '84', '04': '93', '05': '93', '06': '93', '07': '84', '08': '44', '09': '76', '10': '44', '11': '76', '12': '76', '13': '93', '14': '28', '15': '84', '16': '75', '17': '75', '18': '24', '19': '75', '2A': '94', '2B': '94', '21': '27', '22': '53',  '23': '75', '24': '75', '25': '27', '26': '84', '27': '28', '28': '24', '29': '53', '30': '76', '31': '76', '32': '76', '33': '75', '34': '76', '35': '53', '36': '24', '37': '24', '38': '84', '39': '27', '40': '75', '41': '24', '42': '84', '43': '84', '44': '52', '45': '24', '46': '76', '47': '75', '48': '76', '49': '52', '50': '28', '51': '44', '52': '44', '53': '52', '54': '44', '55': '44', '56': '53', '57': '44', '58': '27', '59': '32', '60': '32', '61': '28', '62': '32', '63': '84', '64': '75', '65': '76', '66': '76', '67': '44', '68': '44', '69': '84', '70': '27', '71': '27', '72': '52', '73': '84', '74': '84', '75': '11', '76': '28', '77': '11', '78': '11', '79': '75', '80': '32', '81': '76', '82': '76', '83': '93', '84': '93', '85': '52', '86': '75', '87': '75', '88': '44', '89': '27', '90': '27', '91': '11', '92': '11', '93': '11', '94': '11', '95': '11'}
region_to_name = {'84':'Auvergne-Rhône-Alpes','27':'Bourgogne-Franche-Comté','53':'Bretagne','24':'Centre-Val de Loire','94':'Corse','44':'Grand Est','01':'Guadeloupe','03':'Guyane','32':'Hauts-de-France','11':'Île-de-France','04':'La Réunion','02':'Martinique','28':'Normandie','75':'Nouvelle-Aquitaine','76':'Occitanie','52':'Pays de la Loire','93':"Provence-Alpes-Côte d'Azur"}


In [ ]:
names = pd.read_csv("data/dpt2020.csv", sep=";")
names.head(5)

In [ ]:
# some basic cleaning
names = names[names.annais != "XXXX"]
names["preusuel"] = names["preusuel"].astype(str)
names["annais"] = names["annais"].astype(int)
names["region"] = names["dpt"].map(departement_to_region)
names.drop(columns=["dpt"], inplace=True)

In [ ]:
names_1986 = names[names.annais == 1986]
names_1986.head(5)

In [ ]:
reduced_names_1986 = names_1986.groupby(by=["region", "preusuel", "sexe",  "annais", ]).sum().reset_index().sort_values(["region", "nombre"], ascending=[True, False]) 
reduced_names_1986 = reduced_names_1986[reduced_names_1986.preusuel != "_PRENOMS_RARES"]
reduced_names_1986["nom"] = reduced_names_1986["region"].map(region_to_name)
reduced_names_1986.rename(columns = {"region": "code_insee"}, inplace=True)
reduced_names_1986

In [ ]:
gdf = gpd.read_file('regions/regions-20180101.shp')
gdf["centroid"] = gdf["geometry"].centroid
gdf["x"] = gdf.centroid.map(lambda p: p.x)
gdf["y"] = gdf.centroid.map(lambda p: p.y)
gdf.drop(columns=["centroid"], inplace=True)

In [ ]:
gdf

In [ ]:
region_count = reduced_names_1986.groupby(["code_insee"]).count().reset_index()[["code_insee", "preusuel"]]
region_count.columns = ["code_insee", "Count of unique names"]
region_merge = pd.merge(gdf,region_count, on="code_insee")
region_merge.head(5)

In [ ]:
multi = alt.selection_multi(fields=["code_insee", "nom"], bind='legend')

color = alt.condition(multi,
                      alt.Color('Count of unique names', type='quantitative',
                                scale=alt.Scale(scheme='oranges')),
                      alt.value('lightgray'))

hover = alt.selection(type='single', on='mouseover', nearest=True,
                      fields=['x', 'y'])


region_map = alt.Chart(region_merge[~region_merge.code_insee.isin(["04", "02", "01", "03", "06"])]).mark_geoshape(
    stroke='grey'
).encode(
    color=color,
    tooltip=["nom", 'Count of unique names']
).add_selection(
    multi
).properties(
    title="First name diversity by region",
    width=550,
    height=700
)


labels = alt.Chart(region_merge[~region_merge.code_insee.isin(["04", "02", "01", "03", "06"])]).mark_text().encode(
    longitude='x',
    latitude='y',
    text='nom',
    size=alt.value(12),
    opacity=alt.value(0.8)
)

hist_popularity = alt.Chart(reduced_names_1986.groupby(by="code_insee").head(15).reset_index()).mark_bar().encode(
    x = alt.X("preusuel:N", title="First name", sort="-y"),
    y = alt.Y('nombre', title="Total count"),
    color = alt.Color('nom') 
).properties(
    title="First name popularity for selected regions",
    width=550,
    height=700,
).transform_filter(multi)

region_map + labels | hist_popularity